In [2]:
import os
from typing import TypedDict
import time
from langgraph.graph import StateGraph, END

#### Conditional routing

 ╔════════════════════════════════════════╗
 ║     Conditional Routing Flow          ║
 ╚════════════════════════════════════════╝

       [START]
          │
          ▼
    ┌───────────┐
    │  analyze  │ Checks query length
    └─────┬─────┘ Sets: query_length
          │
     ┌────┴────┐
     │ router()│ Returns string based on
     └────┬────┘ state["query_length"]
          │
    Returns: "quick" or "detailed"
          │
    ┌─────┴─────┐
    ▼           ▼
 ┌──────┐   ┌──────────┐
 │quick │   │ detailed │
 │(<20) │   │  (>=20)  │
 └───┬──┘   └────┬─────┘
     ▼           ▼
   [END]       [END]

 CRITICAL: Router returns MUST match dict keys!
 ┌─────────────────────────────────────┐
 │ router() returns → dict keys match  │
 ├─────────────────────────────────────┤
 │ "quick"         → "quick": "quick"  │
 │ "detailed"      → "detailed": "..." │
 └─────────────────────────────────────┘

In [3]:
class State(TypedDict): 
    query: str
    query_length: str
    response: str 

In [5]:
def analyze_node(state: State):
    print("   🔄 Analyzing query legnth...")
    time.sleep(2)
    length = "short" if len(state['query']) < 20 else "long"
    return {"query length":length}

def router(state:State):
    """Decide which path to take based in query length"""
    if state['query_length'] == 'short':
        return "quick"
    return 'detailed'

def quick_response_node(state: State):
    """Provide a quick response"""
    print(" 🔄 Processing a quick response")
    time.sleep(2)
    response = f"Quick answer: {state['query'][:20]}..."
    return {"response": response}

def detailed_response_node(state: State):
    """Provide a detailed response"""
    print(" 🔄 Processing a detailed analysis")
    time.sleep(2)
    response = f"Detailed analysis: Let me thoroughly explain '{state['query']}'"
    return {"response": response}

print ("Building conditional routing graph\n")
workflow = StateGraph(State)

workflow.add_node("analyze", analyze_node)
workflow.add_node("quick", quick_response_node)
workflow.add_node("detailed", detailed_response_node)

workflow.set_entry_point("analyze")
workflow.add_conditional_edges(
    "analyze",
    router,
    {
        "quick": "quick",
        "detailed": "detailed"
    }
)

# Both paths lead to END
workflow.add_edge("quick", END)
workflow.add_edge("detailed", END)

app = workflow.compile()

Building conditional routing graph



In [6]:
# Test with short query
print("="*50)
print("Test 1: short query")
print("="*40)
result1 = app.invoke({
    "query": "What is Python",
    "query_length": "",
    "response": ""
})

print(f"Query: '{result1['query']}'")
print(f"Route taken: {result1['query_length']}")
print(f"Response: {result1['response']}")

Test 1: short query
   🔄 Analyzing query legnth...
 🔄 Processing a detailed analysis
Query: 'What is Python'
Route taken: 
Response: Detailed analysis: Let me thoroughly explain 'What is Python'
